In [1]:
import re

In [2]:
with open('structure_tree.txt', 'r') as f:
    lines = f.readlines()
    lines = [l.rstrip() for l in lines if len(l) > 0]
    
    current = {}
    structure_list = {}
    
    for l in lines:
        if l.startswith('          - '):
            level = 3
            text = l.strip()[2:]
        elif l.startswith('     - '):
            level = 2
            text = l.strip()[2:]
        elif l.startswith('     '):
            level = 1
            text = l.strip()
        else:
            level = 0
            text = l.strip()
            
        if len(text) == 0:
            continue
        
        try:
            res = re.findall('^(.+?)(:\s?(.*))?$', text)[0]
        except:
            print 'text ERROR:', text
            raise
            
        name, _, desc = res
        
        if '[BEST]' in desc:
            goodness = 'best'
        elif '[GOOD]' in desc:
            goodness = 'good'
        else:
            goodness = None
            
        
        current[level] = name
        
        try:
            fullname, abbreviation = re.findall('^(.*),\s?(.*?)$', name)[0]
        except:
            fullname = name
            abbreviation = ''
        
        parent = current[level-1] if level > 0 else None
        
        structure_list[name] = {'level': level, 
               'fullname': fullname, 
               'abbr': abbreviation, 
               'description': desc,
                'goodness': goodness,
              'parent': parent,
                'children': []}
        
        if parent is not None:
            structure_list[parent]['children'].append(name)
        

In [4]:
structure_list['brainstem'] = {'fullname': 'brainstem', 
                               'level':-1, 
                               'children': [name for name, s in structure_list.iteritems() if s['level'] == 0]}

In [6]:
import json
json.dump(structure_list, open('structure_tree.json', 'w'))

In [3]:
print 'Best structures:'

for name, s in structure_list.iteritems():
    if s['goodness'] == 'best':
        print s['abbr']
        
print '\n'
print 'Good structures:'
        
for name, s in structure_list.iteritems():
    if s['goodness'] == 'good':
        print s['abbr']

Best structures:
fr
Amb
DC
Tz
3N
Pn
RtTg
SNR
7n
AP
10N
VLL
4N
6N
SNC
7N
12N
pc


Good structures:
Sol
LC
VTA
R
SC
IO
IP
IC
5N
LRt


In [ ]:
def print_dict(node):
    print '\t'*node['level'] + node['fullname']
    if len(node['children']) == 0:
        return
    else:
        for name in node['children']:
            print_dict(structure_list[name])
    
for name, s in structure_list.iteritems():
    if s['level'] == 0:
        print_dict(s)

In [ ]:
def extract_names(d, node, structure_tree_dict):

    if len(node['children']) == 0:
        return
    else:
        for name in node['children']:
            print name
            
            if 'abbr' in structure_tree_dict[name] and len(structure_tree_dict[name]['abbr']) > 0:
                key = structure_tree_dict[name]['fullname'] + ' (' + structure_tree_dict[name]['abbr'] + ')'
            else:
                key = structure_tree_dict[name]['fullname']
            
            if key not in d:
                d[key] = {}
                
            extract_names(d[key], structure_tree_dict[name], structure_tree_dict)

In [ ]:
structure_tree_dict = json.load(open('structure_tree.json'))
structure_tree_names = {'brainstem': {}}

extract_names(structure_tree_names['brainstem'], structure_tree_dict['brainstem'], structure_tree_dict)

In [ ]:
structure_tree_names

In [ ]:
def get_children_names(name):
        
    node = structure_tree_dict[name]
    if 'abbr' in node and len(node['abbr']) > 0:
        key = node['fullname'] + ' (' + node['abbr'] + ')'
    else:
        key = node['fullname']
    return (key, dict([get_children_names(child_name) for child_name in node['children']]))

In [ ]:
structure_tree_names = dict([get_children_names('brainstem')])